In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [2]:
model = load_model('model.h5')

In [3]:
with open ('label_encoder_gender.pkl' , 'rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open ('onehot_encoder_geo.pkl' , 'rb') as file:
    onehot_encoder_geo = pickle.load(file)
    
with open ('scaler.pkl' , 'rb') as file:
    Scaler = pickle.load(file)

In [4]:
Test_Data = [
  {
    'RowNumber': 6253, 'CustomerId': 15687492, 'Surname': 'Anderson', 'CreditScore': 596,
    'Geography': 'Germany', 'Gender': 'Male', 'Age': 32, 'Tenure': 3,
    'Balance': 96709.07, 'NumOfProducts': 2, 'HasCrCard': 0,
    'IsActiveMember': 0, 'EstimatedSalary': 41788.37
  },
  {
    'RowNumber': 4685, 'CustomerId': 15736963, 'Surname': 'Herring', 'CreditScore': 623,
    'Geography': 'France', 'Gender': 'Male', 'Age': 43, 'Tenure': 1,
    'Balance': 0.0, 'NumOfProducts': 2, 'HasCrCard': 1,
    'IsActiveMember': 1, 'EstimatedSalary': 146379.3
  },
  {
    'RowNumber': 1732, 'CustomerId': 15721730, 'Surname': 'Amechi', 'CreditScore': 601,
    'Geography': 'Spain', 'Gender': 'Female', 'Age': 44, 'Tenure': 4,
    'Balance': 0.0, 'NumOfProducts': 2, 'HasCrCard': 1,
    'IsActiveMember': 0, 'EstimatedSalary': 58561.31
  },
  {
    'RowNumber': 4743, 'CustomerId': 15762134, 'Surname': 'Liang', 'CreditScore': 506,
    'Geography': 'Germany', 'Gender': 'Male', 'Age': 59, 'Tenure': 8,
    'Balance': 119152.1, 'NumOfProducts': 2, 'HasCrCard': 1,
    'IsActiveMember': 1, 'EstimatedSalary': 170679.74
  },
  {
    'RowNumber': 4522, 'CustomerId': 15648898, 'Surname': 'Chuang', 'CreditScore': 560,
    'Geography': 'Spain', 'Gender': 'Female', 'Age': 27, 'Tenure': 7,
    'Balance': 124995.98, 'NumOfProducts': 1, 'HasCrCard': 1,
    'IsActiveMember': 1, 'EstimatedSalary': 114669.79
  }
]


In [5]:
Test_Data = pd.DataFrame(Test_Data)
Test_Data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,6253,15687492,Anderson,596,Germany,Male,32,3,96709.07,2,0,0,41788.37
1,4685,15736963,Herring,623,France,Male,43,1,0.00,2,1,1,146379.30
2,1732,15721730,Amechi,601,Spain,Female,44,4,0.00,2,1,0,58561.31
3,4743,15762134,Liang,506,Germany,Male,59,8,119152.10,2,1,1,170679.74
4,4522,15648898,Chuang,560,Spain,Female,27,7,124995.98,1,1,1,114669.79


In [6]:
Test_df = Test_Data.drop(['RowNumber' , 'CustomerId' , 'Surname'], axis=1)

In [7]:
Test_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,596,Germany,Male,32,3,96709.07,2,0,0,41788.37
1,623,France,Male,43,1,0.00,2,1,1,146379.30
2,601,Spain,Female,44,4,0.00,2,1,0,58561.31
3,506,Germany,Male,59,8,119152.10,2,1,1,170679.74
4,560,Spain,Female,27,7,124995.98,1,1,1,114669.79


In [8]:
Test_df['Gender'] = label_encoder_gender.transform(Test_df['Gender'])

In [9]:
Test_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,596,Germany,1,32,3,96709.07,2,0,0,41788.37
1,623,France,1,43,1,0.00,2,1,1,146379.30
2,601,Spain,0,44,4,0.00,2,1,0,58561.31
3,506,Germany,1,59,8,119152.10,2,1,1,170679.74
4,560,Spain,0,27,7,124995.98,1,1,1,114669.79


In [10]:
Test_Geography = onehot_encoder_geo.transform(Test_df[['Geography']]).toarray()
Test_Geography_Df = pd.DataFrame(Test_Geography , columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [11]:
Test_Geography_Df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,1.0,0.0
4,0.0,0.0,1.0


In [12]:
Concated_Df = pd.concat([Test_df.drop('Geography', axis=1), Test_Geography_Df],axis=1)

In [13]:
Concated_Df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,596,1,32,3,96709.07,2,0,0,41788.37,0.0,1.0,0.0
1,623,1,43,1,0.00,2,1,1,146379.30,1.0,0.0,0.0
2,601,0,44,4,0.00,2,1,0,58561.31,0.0,0.0,1.0
3,506,1,59,8,119152.10,2,1,1,170679.74,0.0,1.0,0.0
4,560,0,27,7,124995.98,1,1,1,114669.79,0.0,0.0,1.0


In [14]:
Test_Scaled = Scaler.transform(Concated_Df)

In [15]:
Test_Scaled

array([[-0.57749609,  0.91324755, -0.6557859 , -0.69539349,  0.32993735,
         0.80843615, -1.54035103, -1.02583358, -1.01960511, -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , -1.38944225, -1.21847056,
         0.80843615,  0.64920267,  0.97481699,  0.79888291,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, -0.3483691 , -1.21847056,
         0.80843615,  0.64920267, -1.02583358, -0.72797953, -0.99850112,
        -0.57946723,  1.73494238],
       [-1.51149188,  0.91324755,  1.91116988,  1.03972843,  0.68927246,
         0.80843615,  0.64920267,  0.97481699,  1.22138664, -0.99850112,
         1.72572313, -0.57638802],
       [-0.9510944 , -1.09499335, -1.13114808,  0.69270405,  0.78283876,
        -0.91668767,  0.64920267,  0.97481699,  0.24756011, -0.99850112,
        -0.57946723,  1.73494238]])

In [16]:
model.predict(Test_Scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


array([[0.05722054],
       [0.03153436],
       [0.09396985],
       [0.21100959],
       [0.04757719]], dtype=float32)

In [17]:
Prediction = model.predict(Test_Scaled)
Prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


array([[0.05722054],
       [0.03153436],
       [0.09396985],
       [0.21100959],
       [0.04757719]], dtype=float32)

In [18]:
Concated_Df['Prediction'] = Prediction

In [19]:
Concated_Df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Prediction
0,596,1,32,3,96709.07,2,0,0,41788.37,0.0,1.0,0.0,0.057221
1,623,1,43,1,0.00,2,1,1,146379.30,1.0,0.0,0.0,0.031534
2,601,0,44,4,0.00,2,1,0,58561.31,0.0,0.0,1.0,0.093970
3,506,1,59,8,119152.10,2,1,1,170679.74,0.0,1.0,0.0,0.211010
4,560,0,27,7,124995.98,1,1,1,114669.79,0.0,0.0,1.0,0.047577


In [20]:
Final_Data = pd.concat([Test_Data['Surname'], Concated_Df],axis=1)
Final_Data

,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Prediction
0,Anderson,596,1,32,3,96709.07,2,0,0,41788.37,0.0,1.0,0.0,0.057221
1,Herring,623,1,43,1,0.00,2,1,1,146379.30,1.0,0.0,0.0,0.031534
2,Amechi,601,0,44,4,0.00,2,1,0,58561.31,0.0,0.0,1.0,0.093970
3,Liang,506,1,59,8,119152.10,2,1,1,170679.74,0.0,1.0,0.0,0.211010
4,Chuang,560,0,27,7,124995.98,1,1,1,114669.79,0.0,0.0,1.0,0.047577


In [21]:
for index, row in Final_Data.iterrows():
    if row['Prediction'] > 0.5:
        print(f"{row['Surname']} has the intention to leave.")
    else:
        print(f"{row['Surname']} has not the intention to leave.")

Anderson has not the intention to leave.
Herring has not the intention to leave.
Amechi has not the intention to leave.
Liang has not the intention to leave.
Chuang has not the intention to leave.
